Association rules analysis is a technique to uncover how items are associated to each other. 

Association rules are normally written like this: {Diapers} -> {Beer} which means that there is a strong relationship between customers that purchased diapers and also purchased beer in the same transaction.

In the above example, the {Diaper} is the antecedent and the {Beer} is the consequent. Both antecedents and consequents can have multiple items. In other words, {Diaper, Gum} -> {Beer, Chips} is a valid rule.

There are three common ways to measure association.

__Support__
This says how popular an itemset is, as measured by the proportion of transactions in which an itemset appears. 

Support is the relative frequency that the rules show up. In many instances, you may want to look for high support in order to make sure it is a useful relationship. However, there may be instances where a low support is useful if you are trying to find “hidden” relationships.

__Confidence__ is a measure of the reliability of the rule. A confidence of .5 in the above example would mean that in 50% of the cases where Diaper and Gum were purchased, the purchase also included Beer and Chips. For product recommendation, a 50% confidence may be perfectly acceptable but in a medical situation, this level may not be high enough.

__Lift__ is the ratio of the observed support to that expected if the two rules were independent (see wikipedia). The basic rule of thumb is that a lift value close to 1 means the rules were completely independent. Lift values > 1 are generally more “interesting” and could be indicative of a useful rule pattern.

In [1]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
df = pd.read_excel('http://archive.ics.uci.edu/ml/machine-learning-databases/00352/Online%20Retail.xlsx')
df.head()

In [12]:
df.shape

(532621, 8)

By default, apriori returns the column indices of the items, which may be useful in downstream operations such as association rule mining. For better readability, we can set use_colnames=True to convert these integer values into the respective item names:

#### cleanup

- some of the descriptions have spaces that need to be removed. 
- drop the rows that don’t have invoice numbers and 
- remove the credit transactions (those with invoice numbers containing C).

In [11]:
df['Description'] = df['Description'].str.strip()

df.dropna(axis=0, subset=['InvoiceNo'], inplace=True)

df['InvoiceNo']   = df['InvoiceNo'].astype('str')

df = df[~df['InvoiceNo'].str.contains('C')]

In [15]:
df[df['Country'] =="France"].groupby(['InvoiceNo', 'Description']).groups

{('536370', 'ALARM CLOCK BAKELIKE GREEN'): Int64Index([28], dtype='int64'),
 ('536370', 'ALARM CLOCK BAKELIKE PINK'): Int64Index([26], dtype='int64'),
 ('536370', 'ALARM CLOCK BAKELIKE RED'): Int64Index([27], dtype='int64'),
 ('536370',
  'CHARLOTTE BAG DOLLY GIRL DESIGN'): Int64Index([38], dtype='int64'),
 ('536370', 'CIRCUS PARADE LUNCH BOX'): Int64Index([37], dtype='int64'),
 ('536370', 'INFLATABLE POLITICAL GLOBE'): Int64Index([31], dtype='int64'),
 ('536370', 'LUNCH BOX I LOVE LONDON'): Int64Index([36], dtype='int64'),
 ('536370', 'MINI JIGSAW CIRCUS PARADE'): Int64Index([42], dtype='int64'),
 ('536370', 'MINI JIGSAW SPACEBOY'): Int64Index([43], dtype='int64'),
 ('536370', 'MINI PAINT SET VINTAGE'): Int64Index([44], dtype='int64'),
 ('536370',
  'PANDA AND BUNNIES STICKER SHEET'): Int64Index([29], dtype='int64'),
 ('536370', 'POSTAGE'): Int64Index([45], dtype='int64'),
 ('536370', 'RED TOADSTOOL LED NIGHT LIGHT'): Int64Index([39], dtype='int64'),
 ('536370',
  'ROUND SNACK BOXES S

In [19]:
basket  = (df[df['Country'] =="France"]
          .groupby(['InvoiceNo', 'Description'])['Quantity']
          .sum().unstack().reset_index().fillna(0)
          .set_index('InvoiceNo'))

There are a lot of zeros in the data but we also need to make sure any positive values are converted to a 1 and anything less the 0 is set to 0. This step will complete the one hot encoding of the data and remove the postage column

In [20]:
def encode_units(x):
    if x <= 0:
        return 0
    if x >= 1:
        return 1

basket_sets = basket.applymap(encode_units)
basket_sets.drop('POSTAGE', inplace=True, axis=1)

In [21]:
basket_sets

Description,10 COLOUR SPACEBOY PEN,12 COLOURED PARTY BALLOONS,12 EGG HOUSE PAINTED WOOD,12 MESSAGE CARDS WITH ENVELOPES,12 PENCIL SMALL TUBE WOODLAND,12 PENCILS SMALL TUBE RED RETROSPOT,12 PENCILS SMALL TUBE SKULL,12 PENCILS TALL TUBE POSY,12 PENCILS TALL TUBE RED RETROSPOT,12 PENCILS TALL TUBE WOODLAND,...,WRAP VINTAGE PETALS DESIGN,YELLOW COAT RACK PARIS FASHION,YELLOW GIANT GARDEN THERMOMETER,YELLOW SHARK HELICOPTER,ZINC STAR T-LIGHT HOLDER,ZINC FOLKART SLEIGH BELLS,ZINC HERB GARDEN CONTAINER,ZINC METAL HEART DECORATION,ZINC T-LIGHT HOLDER STAR LARGE,ZINC T-LIGHT HOLDER STARS SMALL
InvoiceNo,,,,,,,,,,,,,,,,,,,,,
536370,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536852,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
536974,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537065,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537463,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537468,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537693,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537897,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
537967,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
# Now that the data is structured properly, we can generate frequent 
# item sets that have a support of at least 7% 

frequent_itemsets = apriori(basket_sets, min_support=0.07, use_colnames=True)

In [23]:
# The final step is to generate the rules with their corresponding support, confidence and lift:

rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE GREEN),0.102041,0.096939,0.073980,0.725000,7.478947,0.064088,3.283859
1,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE PINK),0.096939,0.102041,0.073980,0.763158,7.478947,0.064088,3.791383
2,(ALARM CLOCK BAKELIKE GREEN),(ALARM CLOCK BAKELIKE RED),0.096939,0.094388,0.079082,0.815789,8.642959,0.069932,4.916181
3,(ALARM CLOCK BAKELIKE RED),(ALARM CLOCK BAKELIKE GREEN),0.094388,0.096939,0.079082,0.837838,8.642959,0.069932,5.568878
4,(ALARM CLOCK BAKELIKE PINK),(ALARM CLOCK BAKELIKE RED),0.102041,0.094388,0.073980,0.725000,7.681081,0.064348,3.293135


#### Analysis